In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split


In [3]:
with_mask_files = os.listdir("/Users/siddha-book/Desktop/Project/ML Project/Face Mask Detection/data/with_mask")
print(with_mask_files[0:5]) #this will print the first 5 file names

['with_mask_2140.jpg', 'with_mask_1449.jpg', 'with_mask_2626.jpg', 'with_mask_942.jpg', 'with_mask_3538.jpg']


In [4]:
without_mask_files = os.listdir("/Users/siddha-book/Desktop/Project/ML Project/Face Mask Detection/data/without_mask")
print(without_mask_files[0:5]) #this will print the first 5 file names

['without_mask_3593.jpg', 'without_mask_2855.jpg', 'without_mask_1384.jpg', 'without_mask_1390.jpg', 'without_mask_2699.jpg']


In [5]:
# Create Label
# with mask = 1
# without mask = 0

with_mask_labels = [1] * 3725
without_mask_labels = [0] * 3828


In [6]:
#combine them 
labels = with_mask_labels + without_mask_labels

In [7]:
mask_path = 'data/with_mask/'
data = []

for img_file in with_mask_files:

    image = Image.open(mask_path + img_file) #read the file
    image = image.resize((128,128)) #resize it
    image = image.convert('RGB') #convert all to RGB
    image = np.array(image) #convert to numpy array
    data.append(image) #add to the list

unmask_path = 'data/without_mask/'

for img_file in without_mask_files:

    image = Image.open(unmask_path + img_file) #read the file
    image = image.resize((128,128)) #resize it
    image = image.convert('RGB') #convert all to RGB
    image = np.array(image) #convert to numpy array
    data.append(image) #add to the list


/Users/siddha-book/Desktop/Project/ML Project/Face Mask Detection/.venv/lib/python3.11/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [10]:
#convert the data list and label list to numpy array

X = np.array(data)
Y = np.array(labels)

In [11]:
#build the model
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [12]:
#scaling the data
X_train_scaled = X_train/255 
X_test_scaled = X_test/255

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [14]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [20]:
# Convert data to PyTorch tensors
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).permute(0, 3, 1, 2)  # (batch_size, channels, height, width)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).permute(0, 3, 1, 2)    # (batch_size, channels, height, width)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)  # Use long for classification labels
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [21]:
for inputs, labels in train_loader:
    print(inputs.shape)  # Should print: torch.Size([batch_size, 3, 128, 128])
    break

torch.Size([32, 3, 128, 128])


In [16]:
class FaceMaskModel(nn.Module):
    def __init__(self):
        super(FaceMaskModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Input channels = 3 (RGB)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Adjust based on image size after pooling
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)  # Output layer for 2 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Initialize the model and move it to the device
model = FaceMaskModel().to(device)

In [17]:
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Move data to the device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/5], Loss: 0.3963
Epoch [2/5], Loss: 0.2659
Epoch [3/5], Loss: 0.2252
Epoch [4/5], Loss: 0.1827
Epoch [5/5], Loss: 0.1412


In [23]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 93.98%


In [24]:
torch.save(model.state_dict(), 'face_mask_detection_model.pth')

In [25]:
model = FaceMaskModel().to(device)
model.load_state_dict(torch.load('face_mask_detection_model.pth'))
model.eval()

FaceMaskModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)